Using the BBC collection in question, combine the 1 and -1 classes into one and perform classification using the SVM (linear kernel) method, comparing the subjective class (1 and -1) with the objective class (0). Conduct the research for reasonably similar numbers of posts from each class (e.g., 500 and 500). 

In [5]:
# importing libraries
import pandas as pd
import numpy as np
import tmtoolkit as tm
from sklearn.model_selection import train_test_split
from sklearn import svm

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
import matplotlib.pyplot as plt

import kagglehub
from kagglehub import KaggleDatasetAdapter

c:\Users\paula\OneDrive\Pulpit\git_swiezosc\NLP\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [100]:
pip install kagglehub[pandas-datasets]

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# dataset source: https://www.kaggle.com/datasets/advaypatil/youtube-statistics?resource=download
df = pd.read_csv('Lab_5_docs/comments.csv')

# 0 - negative sentiment
# 1 - neutral sentiment
# 2 - positive sentiment
df = df[['Comment', 'Sentiment']]
df = df.sort_values(by = 'Sentiment').reset_index(drop=True)
df

,Comment,Sentiment
0,1 wrong move and it's all gone.,0.0
1,Just stunning that the party once proud of sta...,0.0
2,"Goldmine or not, accepting this concept as the...",0.0
3,Why do we need a metaverse? Why are we doing t...,0.0
4,imposter syndrome is SO REAL! i think that tie...,0.0
...,...,...
18404,Thank you for stopping by put outlawrp I love ...,2.0
18405,"lol great video icy, you never fail to make me...",2.0
18406,*Important Update* : Most of my videos will no...,2.0
18407,Let’s take a moment to appreciate the other ch...,2.0


In [ ]:
# get comments with strong opinion
df_opinion = pd.concat([df.head(600), df.tail(600)])
df_opinion['Sentiment']=1

df_neutral = df.loc[df['Sentiment']==1].sample(n=1200)

df_full = pd.concat([df_opinion,df_neutral])
# df_full

,Comment,Sentiment
0,1 wrong move and it's all gone.,1.0
1,Just stunning that the party once proud of sta...,1.0
2,"Goldmine or not, accepting this concept as the...",1.0
3,Why do we need a metaverse? Why are we doing t...,1.0
4,imposter syndrome is SO REAL! i think that tie...,1.0
...,...,...
5546,How I learn a language at school. \n*MEMORIZIN...,1.0
2468,O BR simplesmente meteu um churrasquinho de ga...,1.0
2909,Agony of watching this when you're the only sn...,1.0
5152,Muito louco esse som 😎,1.0


In [112]:
df_full.columns

Index(['Comment', 'Sentiment'], dtype='object')

In [110]:
corp = tm.corpus.Corpus(dict(zip(range(2400), df_full.Comment)), language = "en", load_features=[])
tm.corpus.set_document_attr(corp, attrname = "Sentiment", data = dict(zip(corp.doc_labels, df_full.Sentiment)))

tm.corpus.print_summary(corp)

Corpus with 2400 documents in English
> 0 (9 tokens): 1 wrong move and it 's all gone .
> 1 (41 tokens): Just stunning that the party once proud of standin...
> 2 (25 tokens): Goldmine or not , accepting this concept as the fu...
> 3 (26 tokens): Why do we need a metaverse ? Why are we doing this...
> 4 (38 tokens): imposter syndrome is SO REAL ! i think that ties b...
> 5 (14 tokens): I ca n't stress this enough . Pricing does not alw...
> 6 (28 tokens): People are dying out of hunger but some are spendi...
> 7 (15 tokens): When you 're watching this as a " refresher " but ...
> 8 (41 tokens): If I called for the posting of the home address of...
> 9 (51 tokens): for people that do nt understand it , we all know ...
(and 2390 more documents)
total number of tokens: 98574 / vocabulary size: 16279


In [111]:
tm.corpus.filter_clean_tokens(corp, remove_numbers=True)
tm.corpus.to_lowercase(corp)

tm.corpus.print_summary(corp)

KeyError: 401

In [75]:
mat, doc_labels, vocab = tm.corpus.dtm(corp, return_doc_labels=True, return_vocab=True)
z = tm.bow.dtm.dtm_to_dataframe(mat, doc_labels, vocab).T

z

,0,1,2,3,4,5,6,7,8,9,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ability,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
able,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
accept,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
according,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yard,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yeah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
year,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yes,0,0,0,0,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0


In [81]:
ind_words = z.sum(axis = 1) >= 5

print(sum(ind_words))

761


In [77]:
print(mat)

<Compressed Sparse Row sparse matrix of dtype 'int32'
	with 31303 stored elements and shape (1200, 762)>
  Coords	Values
  (0, 35)	1
  (0, 80)	1
  (0, 97)	1
  (0, 101)	1
  (0, 152)	1
  (0, 177)	1
  (0, 194)	1
  (0, 309)	1
  (0, 331)	1
  (0, 346)	1
  (0, 373)	1
  (0, 381)	1
  (0, 415)	1
  (0, 445)	1
  (0, 453)	1
  (0, 486)	1
  (0, 499)	1
  (0, 530)	1
  (0, 544)	1
  (0, 550)	1
  (0, 611)	1
  (0, 613)	1
  (0, 617)	1
  (0, 694)	1
  (0, 717)	1
  :	:
  (1198, 739)	1
  (1199, 14)	1
  (1199, 39)	1
  (1199, 46)	1
  (1199, 102)	1
  (1199, 121)	1
  (1199, 128)	1
  (1199, 199)	1
  (1199, 206)	1
  (1199, 226)	1
  (1199, 240)	1
  (1199, 251)	1
  (1199, 274)	1
  (1199, 279)	1
  (1199, 330)	1
  (1199, 342)	1
  (1199, 355)	1
  (1199, 364)	1
  (1199, 436)	1
  (1199, 497)	1
  (1199, 519)	1
  (1199, 552)	1
  (1199, 620)	1
  (1199, 667)	1
  (1199, 678)	1


In [79]:
mat1 = mat[:,ind_words]


# z1 = tm.bow.dtm.dtm_to_dataframe(mat1, doc_labels, np.array(vocab)[ind_words]).T

# ind_docs = z1.sum(axis = 0) >= 5

# mat2 = mat1[ind_docs,:]

# z2 = tm.bow.dtm.dtm_to_dataframe(mat2, np.array(doc_labels)[ind_docs], np.array(vocab)[ind_words])

# z2.T

AttributeError: 'Series' object has no attribute 'nonzero'